# FRANKFURTER API - Currency Time Series Prediction

# Notebook 2 : Actualización

Documentación:https://www.frankfurter.app/docs/

The Frankfurter API tracks foreign exchange references rates published by the European Central Bank. The data refreshes around 16:00 CET every working day.

In [1]:
import pandas as pd
import numpy as np

import time
import datetime

import requests

from pprint import pprint

## Variables Globales

Variables necesarias para la extracción y organización de la información que nos brinda la API de Frankfurter.

In [2]:
url = "https://api.frankfurter.app"

## Funciones

Función para actualizar el DataFrame desde la última vez que recopiló la información hasta el día actual en el que se realice la actualización.

In [3]:
def update_dataframe(dataframe):
    
    from_currency = dataframe.columns[-2]
    init_date = dataframe["Dates"].iloc[-1]
    fin_date = datetime.datetime.today()
    amount = dataframe.iloc[-1, -2]
    
    days_dif = datetime.datetime.strptime(datetime.datetime.strftime(fin_date, "%Y-%m-%d"), "%Y-%m-%d") - datetime.datetime.strptime(init_date, "%Y-%m-%d")
    
    days_dif = days_dif.days
    
    if days_dif == 0:
        
        print("No hay actualizaciones disponibles.")
        print("La información se actualiza diariamente después de las 17:00 (hora estándar de Europa Central).")
        print("Inténtelo después.\n")
        
        return dataframe
    
    else:
        
        if fin_date.hour < 17:
            
            print("No hay actualizaciones disponibles.")
            print("La información se actualiza diariamente después de las 16:00 (hora estándar de Europa Central).")
            print("Inténtelo después.\n")
        
            return dataframe
        
        else:
            
            values = list()

            dates = list()

            for days in range(1, days_dif + 1):

                updated_values = dict()

                date = datetime.datetime.strptime(init_date, "%Y-%m-%d") + datetime.timedelta(days = days)

                date = datetime.datetime.strftime(date, "%Y-%m-%d")

                endpoint = f"{url}/{date}?amount={amount}&from={from_currency}"

                response = requests.get(endpoint).json()

                for col in dataframe.columns[:-2]:

                    try:

                        updated_values[col] = response["rates"][col]

                    except:

                        updated_values[col] = np.nan

                updated_values["Dates"] = date

                updated_values[from_currency] = amount

                # Convertir el diccionario en un DataFrame
                updated_values_df = pd.DataFrame.from_dict(updated_values, orient = 'index', columns = [dataframe.shape[0]])

                # Concatenar el nuevo DataFrame al existente (axis=0 para apilar en filas)
                dataframe = pd.concat([dataframe, updated_values_df.T], axis = 0, sort = False)

                print(f"-----------{date} completed-----------")

            return dataframe
        
# VALIDAR LA CONDICIÓN DE SI SE REPITEN LOS MISMOS DATOS ENTONCES NO HAY ACTUALIZACIÓN YA QUE AUNQUE SEAN MÁS DE LAS 16:00 PUEDE QUE NO SE HAYA ACTUALIZADO TODAVÍA

## Actualización del DataFrame original

Cargamos el último DataFrame guardado para actualizarlo, si corresponde.

In [4]:
historical = pd.read_parquet("Data/historical.parquet")

In [5]:
historical

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,SGD,SIT,SKK,THB,TRL,TRY,USD,ZAR,EUR,Dates
0,1.9100,NaN,NaN,1.8004,1.6168,NaN,0.58231,35.107,7.4501,15.6466,...,1.9554,189.04,42.991,NaN,372274.0,NaN,1.1789,6.9358,1,1999-01-04
1,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.58230,34.917,7.4495,15.6466,...,1.9655,188.78,42.848,NaN,372816.0,NaN,1.1790,6.7975,1,1999-01-05
2,1.8820,NaN,NaN,1.7711,1.6116,NaN,0.58200,34.850,7.4452,15.6466,...,1.9699,188.70,42.778,NaN,372188.0,NaN,1.1743,6.7307,1,1999-01-06
3,1.8474,NaN,NaN,1.7602,1.6165,NaN,0.58187,34.886,7.4431,15.6466,...,1.9436,188.80,42.765,NaN,370147.0,NaN,1.1632,6.8283,1,1999-01-07
4,1.8406,NaN,NaN,1.7643,1.6138,NaN,0.58187,34.938,7.4433,15.6466,...,1.9537,188.84,42.560,NaN,371830.0,NaN,1.1659,6.7855,1,1999-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9022,1.6498,1.9558,5.1860,1.4409,0.9554,7.7561,NaN,24.496,7.4573,NaN,...,1.4524,NaN,NaN,38.145,NaN,28.751,1.0658,20.2970,1,2023-09-17
9023,1.6575,1.9558,5.1905,1.4400,0.9564,7.7796,NaN,24.406,7.4551,NaN,...,1.4551,NaN,NaN,38.072,NaN,28.811,1.0663,20.2950,1,2023-09-18
9024,1.6566,1.9558,5.1914,1.4373,0.9595,7.8111,NaN,24.432,7.4551,NaN,...,1.4596,NaN,NaN,38.492,NaN,28.960,1.0713,20.2830,1,2023-09-19
9025,1.6498,1.9558,5.1982,1.4366,0.9601,7.8059,NaN,24.388,7.4528,NaN,...,1.4581,NaN,NaN,38.591,NaN,28.934,1.0702,20.1590,1,2023-09-20


In [6]:
%%time

historical = update_dataframe(dataframe = historical)

-----------2023-09-22 completed-----------
-----------2023-09-23 completed-----------
-----------2023-09-24 completed-----------
-----------2023-09-25 completed-----------
-----------2023-09-26 completed-----------
-----------2023-09-27 completed-----------
-----------2023-09-28 completed-----------
-----------2023-09-29 completed-----------
CPU times: total: 93.8 ms
Wall time: 1.15 s


In [7]:
historical

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,SGD,SIT,SKK,THB,TRL,TRY,USD,ZAR,EUR,Dates
0,1.91,NaN,NaN,1.8004,1.6168,NaN,0.58231,35.107,7.4501,15.6466,...,1.9554,189.04,42.991,NaN,372274.0,NaN,1.1789,6.9358,1,1999-01-04
1,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.58230,34.917,7.4495,15.6466,...,1.9655,188.78,42.848,NaN,372816.0,NaN,1.179,6.7975,1,1999-01-05
2,1.882,NaN,NaN,1.7711,1.6116,NaN,0.58200,34.85,7.4452,15.6466,...,1.9699,188.70,42.778,NaN,372188.0,NaN,1.1743,6.7307,1,1999-01-06
3,1.8474,NaN,NaN,1.7602,1.6165,NaN,0.58187,34.886,7.4431,15.6466,...,1.9436,188.80,42.765,NaN,370147.0,NaN,1.1632,6.8283,1,1999-01-07
4,1.8406,NaN,NaN,1.7643,1.6138,NaN,0.58187,34.938,7.4433,15.6466,...,1.9537,188.84,42.560,NaN,371830.0,NaN,1.1659,6.7855,1,1999-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030,1.6544,1.9558,5.2516,1.4311,0.9676,7.7728,NaN,24.377,7.4571,NaN,...,1.4524,NaN,NaN,38.396,NaN,28.931,1.0633,19.941,1,2023-09-25
9031,1.6539,1.9558,5.2721,1.4318,0.9675,7.7517,NaN,24.411,7.4575,NaN,...,1.45,NaN,NaN,38.549,NaN,28.919,1.0605,20.139,1,2023-09-26
9032,1.6541,1.9558,5.2716,1.4255,0.968,7.7031,NaN,24.47,7.4566,NaN,...,1.4449,NaN,NaN,38.541,NaN,28.767,1.0536,20.222,1,2023-09-27
9033,1.6483,1.9558,5.3115,1.4206,0.9678,7.6986,NaN,24.378,7.4558,NaN,...,1.4431,NaN,NaN,38.657,NaN,28.895,1.0539,20.17,1,2023-09-28


Guardo el DataFrame actualizado para entrenar el modelo con los datos más actuales posibles.

In [8]:
historical.to_parquet("Data/historical.parquet")

In [9]:
del historical

In [10]:
################################################## Sigue en FORECASTING ###################################################